In [3]:
import pandas as pd
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score,precision_score,f1_score

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     C:\Users\cardo\AppData\Roaming\nltk_data...
[nltk_data]    | 

In [5]:

df = pd.read_csv('Sentiment_Dataset.csv')
df

,Sentence,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1
...,...,...
2743,I just got bored watching Jessice Lange take h...,0
2744,"Unfortunately, any virtue in this film's produ...",0
2745,"In a word, it is embarrassing.",0
2746,Exceptionally bad!,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Sentence   2748 non-null   object
 1   Sentiment  2748 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 43.1+ KB


In [7]:
def preprocess_text(text):
    # Convertir el texto a minúscula
    text = text.lower()

    # Tokenizar el texto
    tokens = word_tokenize(text)

    # Eliminar las stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]

    # Lematizar los tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Unir los tokens para obtener el texto procesado
    processed_text = ' '.join(tokens)

    return processed_text

In [8]:
df['Sentence']=df['Sentence'].astype('string')

In [9]:
dfc=df.copy()

In [10]:
dfc['Sentence']=df['Sentence'].apply(preprocess_text)

In [11]:
dfc.head()

,Sentence,Sentiment
0,wow ... loved place .,1
1,crust good .,0
2,tasty texture nasty .,0
3,stopped late may bank holiday rick steve recom...,1
4,selection menu great price .,1


In [12]:
from sklearn.preprocessing import LabelEncoder

# Codificación de etiquetas
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(df['Sentiment'])

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(dfc['Sentence'], y_encoded, test_size=0.2, random_state=42)

print(f"Training size: {len(X_train)}")
print(f"Test size: {len(X_test)}")

Training size: 2198
Test size: 550


In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Definir el tokenizador
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# Convertir texto a secuencias de números
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# Hacer padding a las secuencias
maxlen = 100
train_padded = pad_sequences(train_sequences, maxlen=maxlen, padding='post', truncating='post')
test_padded = pad_sequences(test_sequences, maxlen=maxlen, padding='post', truncating='post')


LSTM MODEL

In [14]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Definir el modelo
model = Sequential()

# Capa de Embedding
model.add(Embedding(input_dim=10000,
                    output_dim=128,
                    input_length=100))

# Capa LSTM
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))

# Segunda capa LSTM
model.add(LSTM(32))
model.add(Dropout(0.2))

# Capa Densa de Salida
model.add(Dense(1, activation='sigmoid'))

# Compilar el modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 100, 64)           49408     
                                                                 
 dropout (Dropout)           (None, 100, 64)           0         
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 1341857 (5.12 MB)
Trainable params: 1341

In [15]:
history = model.fit(train_padded, y_train, epochs=10, batch_size=128, validation_split=0.2)


Epoch 1/10


14/14 [==============================] - 9s 328ms/step - loss: 0.6942 - accuracy: 0.4966 - val_loss: 0.6925 - val_accuracy: 0.5250
Epoch 2/10
14/14 [==============================] - 3s 230ms/step - loss: 0.6934 - accuracy: 0.4943 - val_loss: 0.6928 - val_accuracy: 0.5250
Epoch 3/10
14/14 [==============================] - 3s 236ms/step - loss: 0.6929 - accuracy: 0.5000 - val_loss: 0.6927 - val_accuracy: 0.5250
Epoch 4/10
14/14 [==============================] - 3s 236ms/step - loss: 0.6934 - accuracy: 0.5080 - val_loss: 0.6928 - val_accuracy: 0.5227
Epoch 5/10
14/14 [==============================] - 3s 235ms/step - loss: 0.6938 - accuracy: 0.4960 - val_loss: 0.6924 - val_accuracy: 0.5250
Epoch 6/10
14/14 [==============================] - 3s 230ms/step - loss: 0.6935 - accuracy: 0.4903 - val_loss: 0.6934 - val_accuracy: 0.4750
Epoch 7/10
14/14 [==============================] - 3s 237ms/step - loss: 0.6927 - accuracy: 0.5250 - val_loss: 0.6925 - val_accuracy: 0.5227
Epoc

In [16]:
from sklearn.metrics import classification_report, accuracy_score, cohen_kappa_score

# Predecir las etiquetas para el conjunto de prueba
y_pred = (model.predict(test_padded) > 0.5).astype("int32")

# Calcular y mostrar las métricas
print(classification_report(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))
print("Kappa Score:", cohen_kappa_score(y_test, y_pred))


18/18 [==============================] - 1s 31ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       285
           1       0.48      1.00      0.65       265

    accuracy                           0.48       550
   macro avg       0.24      0.50      0.33       550
weighted avg       0.23      0.48      0.31       550

Accuracy Score: 0.4818181818181818
Kappa Score: 0.0


C:\Users\cardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cardo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1469

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

def create_model(optimizer='adam', output_dim=128, lstm_units=64, dropout_rate=0.2):
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=output_dim, input_length=100))
    model.add(LSTM(lstm_units, return_sequences=True))
    model.add(Dropout(dropout_rate))
    model.add(LSTM(int(lstm_units / 2)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(model=create_model, epochs=10, batch_size=128)


In [22]:
from scikeras.wrappers import KerasClassifier

model = KerasClassifier(model=create_model)


In [23]:
param_grid = {
    'model__optimizer': ['adam', 'rmsprop'],
    'model__output_dim': [128, 256],
    'model__lstm_units': [64, 128],
    'model__dropout_rate': [0.2, 0.3],
    'batch_size': [32, 64],
    'epochs': [5, 10] #esta prueba tarda mucho, bajar los valores si se quiere una prueba rapida
}


In [24]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)
grid_result = grid.fit(train_padded, y_train)  


Epoch 1/5
46/46 [==============================] - 9s 108ms/step - loss: 0.6950 - accuracy: 0.4853
Epoch 2/5
46/46 [==============================] - 5s 105ms/step - loss: 0.6935 - accuracy: 0.5044
Epoch 3/5
46/46 [==============================] - 5s 109ms/step - loss: 0.6944 - accuracy: 0.4983
Epoch 4/5
46/46 [==============================] - 5s 108ms/step - loss: 0.6933 - accuracy: 0.5119
Epoch 5/5
23/23 [==============================] - 1s 27ms/step
Epoch 1/5
46/46 [==============================] - 8s 103ms/step - loss: 0.6949 - accuracy: 0.5051
Epoch 2/5
46/46 [==============================] - 5s 103ms/step - loss: 0.6939 - accuracy: 0.4915
Epoch 3/5
46/46 [==============================] - 5s 108ms/step - loss: 0.6930 - accuracy: 0.5167
Epoch 4/5
46/46 [==============================] - 5s 107ms/step - loss: 0.6937 - accuracy: 0.5003
Epoch 5/5
23/23 [==============================] - 1s 25ms/step
Epoch 1/5
46/46 [==============================] - 9s 111ms/step - loss: 0.6941 

In [25]:
print("Mejor: %f usando %s" % (grid_result.best_score_, grid_result.best_params_))


Mejor: 0.510464 usando {'batch_size': 32, 'epochs': 10, 'model__dropout_rate': 0.2, 'model__lstm_units': 64, 'model__optimizer': 'adam', 'model__output_dim': 128}


DummyClassifier Model

In [26]:
#DummyClassifier
from sklearn.dummy import DummyClassifier

In [27]:
classifier = DummyClassifier(strategy="most_frequent")

In [28]:
classifier.fit(train_sequences, y_train)

DummyClassifier(strategy='most_frequent')

In [29]:
predictions = classifier.predict(test_sequences)

In [30]:
print("Accuracy=",accuracy_score(y_test,predictions))
print("Kappa=",cohen_kappa_score(y_test,predictions))
print("Precision=",precision_score(y_test, predictions,average='micro'))
print("F1-score=",f1_score(y_test,predictions, average='micro'))

Accuracy= 0.4818181818181818
Kappa= 0.0
Precision= 0.4818181818181818
F1-score= 0.4818181818181818


Vanilla RNN Model

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

def RNN():
    model = Sequential()
    model.add(Embedding(input_dim=10000, output_dim=32, input_length=100))  # Capa de Embedding
    model.add(SimpleRNN(64))  # Capa RNN con 64 unidades
    model.add(Dense(1, activation='sigmoid'))  # Capa de salida para clasificación binaria

    # Compilar el modelo
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Crear el modelo RNN
model = RNN()


In [35]:
# Entrenar el modelo
model = RNN()
model.fit(train_padded, y_train, epochs=10, batch_size=128)

# Hacer predicciones sobre el conjunto de prueba
y_pred = model.predict(test_padded)
y_pred = (y_pred > 0.5).astype(int)

Epoch 1/10
18/18 [==============================] - 2s 29ms/step - loss: 0.6876 - accuracy: 0.5350
Epoch 2/10
18/18 [==============================] - 0s 26ms/step - loss: 0.5838 - accuracy: 0.7234
Epoch 3/10
18/18 [==============================] - 0s 26ms/step - loss: 0.4154 - accuracy: 0.8412
Epoch 4/10
18/18 [==============================] - 0s 25ms/step - loss: 0.2728 - accuracy: 0.9140
Epoch 5/10
18/18 [==============================] - 0s 27ms/step - loss: 0.2859 - accuracy: 0.9017
Epoch 6/10
18/18 [==============================] - 0s 27ms/step - loss: 0.2193 - accuracy: 0.9227
Epoch 7/10
18/18 [==============================] - 0s 25ms/step - loss: 0.1321 - accuracy: 0.9654
Epoch 8/10
18/18 [==============================] - 0s 25ms/step - loss: 0.0971 - accuracy: 0.9759
Epoch 9/10
18/18 [==============================] - 0s 26ms/step - loss: 0.0795 - accuracy: 0.9768
Epoch 10/10
18/18 [==============================] - 0s 7ms/step


In [37]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
kappa = cohen_kappa_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")
print(f"Kappa Score: {kappa}")

Accuracy: 0.7072727272727273
Precision: 0.7363636363636363
Recall: 0.6113207547169811
F1 Score: 0.6680412371134021
Kappa Score: 0.41025641025641024
